In [1]:
pip install optuna scikit-learn

     |████████████████████████████████| 286kB 8.9MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 1.1MB 14.4MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 112kB 34.6MB/s 
     |████████████████████████████████| 133kB 29.4MB/s 
     |████████████████████████████████| 481kB 27.4MB/s 
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=cbd470e749f4856db984b672c2f0e3f5f4663f0bb8a2c20b8b6ef30789b95cec
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=0cb70e6b15c2cb1b49c3e60758ad12e4f534974423b5ccec3166f45043f18606
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha

In [15]:
from functools import partial

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn import datasets


def objective(X, y, trial):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear','rbf','poly','sigmoid']),
        'C': trial.suggest_loguniform('C', 1e-3, 1e+3),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 1e+3)
    }
    model = SVC(**params)

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X, y=y, cv=kf)
    return 1.0 - scores['test_score'].mean()


dataset = datasets.load_digits()
X, y = dataset.data, dataset.target
f = partial(objective, X, y)
study = optuna.create_study()
study.optimize(f, n_trials=100)
print('params:', study.best_params)

[I 2021-01-31 05:18:10,850] A new study created in memory with name: no-name-f0f8be6c-b31e-4f16-84c2-47eef4f8c15f
[I 2021-01-31 05:18:13,442] Trial 0 finished with value: 0.8597214484679666 and parameters: {'kernel': 'rbf', 'C': 122.42611570450612, 'gamma': 2.224351522899567}. Best is trial 0 with value: 0.8597214484679666.
[I 2021-01-31 05:18:13,715] Trial 1 finished with value: 0.020037140204271053 and parameters: {'kernel': 'linear', 'C': 0.06887349505123787, 'gamma': 125.14052235857973}. Best is trial 1 with value: 0.020037140204271053.
[I 2021-01-31 05:18:14,047] Trial 2 finished with value: 0.012800990405447243 and parameters: {'kernel': 'poly', 'C': 0.22437503311495857, 'gamma': 0.44815873932288414}. Best is trial 2 with value: 0.012800990405447243.
[I 2021-01-31 05:18:16,432] Trial 3 finished with value: 0.8987186629526462 and parameters: {'kernel': 'rbf', 'C': 0.001904158995987334, 'gamma': 23.615529053740097}. Best is trial 2 with value: 0.012800990405447243.
[I 2021-01-31 05

params: {'kernel': 'poly', 'C': 0.22437503311495857, 'gamma': 0.44815873932288414}
